In [1]:
import numpy as np
from modules import bayesianL
from modules import cubicharmonics
import matplotlib.pyplot as plt
import copy

In [2]:
#Carico i dati in data e li ordino in sdata e grid

data=np.transpose(np.loadtxt('../thermopolarizationresponse.out'))

sdata, grid = bayesianL.generatesorteddata(data, 500)

[0. 1. 0.] [0. 0. 1.] [1. 0. 0.]


In [3]:
#Medio i dati su tutti i punti equivalenti per simmetria cubica e 
#ritorno la lista dei nuovi dati 'simmetrizzati' in gplot e sd
C=bayesianL.cubicarray(list(grid), True)
dic, dics=bayesianL.datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot, datasigmaplot=bayesianL.datiplot(C, dic, dics)
sd=np.array([dataplot, datasigmaplot])

numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118


In [4]:
#Calcolo il fit bayesiano con glot e sd come test
N=35
mN, SN, y_infer, sy_infer=\
bayesianL.bayesianpol(gplot, sd, 10, N, 0.000005, gplot, ifprint= True, ifwarning= True)

parametri ottimali [ 0.00277506  0.01147751 -0.15971614  0.54601284  0.11702633  0.33892295
  0.166861   -0.30763027 -0.11112603 -0.28677292  0.30907324  0.05568825
  0.31167945 -0.18556506 -0.08065821 -0.18297158  0.05108597  0.1673547
  0.05984718  0.02241794 -0.15179457 -0.00602741 -0.01944471  0.0536847
 -0.00345853]
numero di armoniche cubiche 14
valore a kmin inferito 0.0028384367993449823 bias 0.002775063061096094 dato a kmin 0.0029604125925925813
determinante matrice delle armoniche cubiche ridotte: 7.853103235256358e+145


In [5]:
#Plotto il fit e i dati (gplot e sd)
C=bayesianL.cubicarray(list(grid), True)
dic, dics=bayesianL.datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot, datasigmaplot=bayesianL.datiplot(C, dic, dics)
%matplotlib widget

plt.fill_between(np.linalg.norm(gplot[:N], axis=1), y_infer+np.sqrt(sy_infer), y_infer-np.sqrt(sy_infer),  alpha=0.3, color='red')
plt.plot(np.linalg.norm(gplot[:N], axis=1), y_infer, '.-', color='red')
plt.errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot, fmt='.')

numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [6]:
#Calcolo il best fit su gplot e sd con 30 dati

mN, SN, y_infer, sy_infer, spar, log_evidence_vP=\
bayesianL.bestfit(gplot, sd, N, gplot, ifprintbestfit= False, ifprintfinal= True)

parametri ottimali [0.00282806 0.0015248 ]
numero di armoniche cubiche 2
valore a kmin inferito 0.002839632503261264 bias 0.0028280641703593273 dato a kmin 0.0029604125925925813
determinante matrice delle armoniche cubiche ridotte: 4.082301903684676e+19
grado ottimale 4 grado massimo tentato 28


In [7]:
#Plotto la funzione di evidence che ho massimizzato per il best fit nella cella precedente
%matplotlib widget
plt.plot(range(2,(len(log_evidence_vP)+1)*2, 2), log_evidence_vP, '.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
#Plotto il best fit e i dati (gplot e sd)
C=bayesianL.cubicarray(list(grid), True)
dic, dics=bayesianL.datadicG(data[1], data[2],cubicharmonics.Gvecgenerateall(500)[1:])
gplot, dataplot, datasigmaplot=bayesianL.datiplot(C, dic, dics)
%matplotlib widget

plt.fill_between(np.linalg.norm(gplot[:N], axis=1), y_infer+np.sqrt(sy_infer), y_infer-np.sqrt(sy_infer),  alpha=0.3, color='red')
plt.plot(np.linalg.norm(gplot[:N], axis=1), y_infer, '.-', color='red')
plt.errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot, fmt='.')

numero di elementi:  499
numero cicli teornicamente necessari 373501
numero cicli compiuti 381
numero di elementi non equivalenti per simmetria cubica:  118


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ErrorbarContainer object of 3 artists>

In [9]:
#Calcolo il best fit includendo sempre piu' punti 
predb=[]
spredb=[]
Npointsb=[]
for N in range(10, 101):
    mN, SN, y_infer, sy_infer, spar, log_evidence_vP=\
    bayesianL.bestfit(gplot, sd, N, gplot, ifprintbestfit=False, ifprintfinal=False)
    Npointsb.append(np.linalg.norm(gplot[N]))
    if (N-10)%(90//10)==0: print(int((N-10)/90*100), '%')
    predb.append(mN[0])
    spredb.append(SN[0,0])

0 %
10 %
20 %
30 %
40 %
50 %
60 %
70 %
80 %
90 %
100 %


In [10]:
#Calcolo il model averaging su gplot  e sd con 30 punti
N=20
mr, sr, logevidence=\
bayesianL.bayesianmodelprediction(gplot, sd, N, gplot, ifprintmodpred=False, ifprintfinal=True)

model average prediction 0.002877225790819467 7.419861103438519e-06


In [11]:
#Calcolo il model averaging includendo sempre piu' punti 
preda=[]
spreda=[]
Npointsa=[]
for N in range(10, 101):
    
    mr, sr, logevidence=\
    bayesianL.bayesianmodelprediction(gplot, sd, N, gplot, ifprintmodpred=False, ifprintfinal=False)
    Npointsa.append(np.linalg.norm(gplot[N]))
    if (N-10)%(90//10)==0: print(int((N-10)/90*100), '%')
    preda.append(mr)
    spreda.append(sr)

0 %
10 %
20 %
30 %
40 %
50 %
60 %
70 %
80 %
90 %
100 %


In [32]:
%matplotlib widget
fig, ax=plt.subplots(2,constrained_layout=True, figsize=[9,9])


#ax[0].set_title('Bayesian model averaging prediction at k=0 with data in range $[: k_{max}]$')
#
#ax[0].errorbar(np.array(Npointsa), np.array(preda), np.sqrt(np.array(spreda)), fmt='.')
#    
#ax[0].set_xlabel('$k_{max} (nm^{-1})$')
#ax[0].set_ylabel('prediction at k=0')
#ax[0].legend()
ax[0].set_title('Bayesian best model prediction at k=0 with data in range $[: k_{max}]$')

ax[0].errorbar(np.array(Npointsb), np.array(predb), np.sqrt(np.array(spredb)), fmt='.')

ax[0].set_xlabel('$k_{max} (nm^{-1})$')
ax[0].set_ylabel('prediction at k=0')
ax[0].legend()
ax[1].set_title('Data')

ax[1].errorbar(np.linalg.norm(gplot, axis=1), dataplot,datasigmaplot, fmt='.', label='3375 water molecules at 300k 30ns') 

ax[1].legend()

ax[1].set_xlabel(r'k ($nm^{-1}$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'k ($nm^{-1}$)')